<a href="https://colab.research.google.com/github/hanzhang690/hanRepository/blob/master/VerticalSep_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install and load the required Libraries 

In [1]:
#Install libraries
!pip install geopandas
!pip install geopy
!pip install shapely 
!pip install pandasql

#Needed on Google Colab not AWS
!pip install dask[dataframe]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 4.9 MB/s 
     |████████████████████████████████| 6.3 MB 4.1 MB/s 
     |████████████████████████████████| 16.7 MB 3.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=cdf4c5a58be50d245c008aaf8096f36d8cbc9a0d6cdeec60dfe0fa5720f469b0
  Stored in directory: /root/.cache/pip/wheels/5c/4b/ec/41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 140 

In [2]:
#Import Libraries
import pandas as pd
import geopandas as ps
import geopy as gy
import shapely as sy
import dask.dataframe as dd
import pandasql as pq
from shapely.geometry import Point, Polygon
import numpy as np
from math import radians, cos, sin, asin, sqrt

#Specific libraries to Google Colab -- will not be needed in final product
from google.colab import files
import io
import os
import sys
import re

Upload Data (RawData_v2.csv file)

In [3]:
#Upload the full datafile specific to Google Drive, AWS will be different 

uploadedfile = files.upload()
rawData_df = pd.read_csv(io.BytesIO(uploadedfile['RawData_v2.csv']))
print(f'Total record count : ',len(rawData_df.index))

Saving RawData_v2.csv to RawData_v2.csv


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Total record count :  33779


Data clean up / transformation

In [4]:
#Function to filter out the needed attribues, rename, change flight level scale, and filter for only those flights at or above flight level 240
def filterAttributes():
    #New dataframe with selected attributes from the raw data
    airspaceData_df = rawData_df[["FRN73TMRPDateTimeOfMessageRec","FRN131HRPWCFloatingPointLat","FRN131HRPWCFloatingPointLong",
                     "FRN145FLFlightLevel", "FRN170TITargetId","RESHSelectedHeading","FRN80TATargetAddress",
                     "FRN161TNTrackNumber"]]

    #Rename columns to make it easier to read
    airspaceData_df = airspaceData_df.rename(columns={'FRN73TMRPDateTimeOfMessageRec': 'DateTime', 
                                                      'FRN131HRPWCFloatingPointLat': "Latitude", 
                                                      'FRN131HRPWCFloatingPointLong': "Longitude", 
                                                      'FRN145FLFlightLevel': "FlightLevel", 
                                                      'FRN170TITargetId': "TargetID", 
                                                      'RESHSelectedHeading': "SelectedHeading", 
                                                      'FRN80TATargetAddress': "TargetAddress",
                                                      'FRN161TNTrackNumber': "TrackNumber"})
    
    # Remove anything less than 240 flight level 
    airspaceData_df = airspaceData_df[(airspaceData_df['FlightLevel'] >= 240)]
    
    #Change flight level scale to feet (FL1 = 100 ft)
    airspaceData_df['FlightLevel'] = airspaceData_df['FlightLevel'].apply(lambda x: x * 100)
    
    airspaceData = airspaceData_df
    
    return airspaceData

In [5]:
#Function to keep only the first 5 seconds of each minute to speed up processing 

def firstFiveSeconds():
    
    #Set the dataframe that will be altered through this block of code
    global airspaceData
    
    char = ['T','Z']
    for x in char:
        airspaceData["DateTime"] = airspaceData["DateTime"].str.replace( x ," ")

    # Formatted Datetime
    airspaceData["DateTime"] = pd.to_datetime(airspaceData["DateTime"], format="%Y-%m-%d %H:%M:%S")
    
    # Create 4 new columns for Hour, Minute, Second and Microsecond
    airspaceData["Hour"] = airspaceData["DateTime"].dt.hour
    airspaceData["Minute"] = airspaceData["DateTime"].dt.minute
    airspaceData["Second"] = airspaceData["DateTime"].dt.second
    airspaceData["microSecond"] = airspaceData["DateTime"].dt.microsecond
    
    # Reorder columns
    airspaceData = airspaceData[["DateTime","Hour","Minute","Second","microSecond","Latitude","Longitude","FlightLevel",
                                   "TargetID","SelectedHeading","TargetAddress",
                                   "TrackNumber"]]
    
    rawAircraftData = airspaceData
    
    #Keep only records for the first 5 seconds to speed up processing time 
    airspaceData = airspaceData[(airspaceData['Second'] < 5)]
    
    return (rawAircraftData)

In [6]:
#Function to filter out anything in the Hawaii airspace

def removeHISpace():
    
    #Set the dataframe that will be altered through this block of code
    global airspaceData
    
    #Coordinates for Hawaii airspace
    v0 = (26.14472222, -158.62194444) 
    v1 = (26.105, -160.63166667)
    v2 = (25.67611111, -161.69111111)
    v3 = (25.05666667, -162.64972222)
    v4 = (24.16889, -163.26638889)
    v5 = (23.25833, -163.855)
    v6 = (22.20555556, -163.91444444)

    #Select the correct v7 depending on what you are testing
    v7 = (33.10266389, 130.47177778) #Incorrect point to use during development
    #v7 = (21.1511111, -163.9144444) #Correct point to use when going live
    
    v8 = (20.11666667, -163.3)
    v9 = (19.65805556,-162.69944444)
    v10 = (19.415, -162.38361111)
    v11 = (18.40777778, -160.81416667)
    v12 = (18.0525, -160.26972222)
    v13 = (17.75583333, -159.53888889)
    v14 = (17.17055556, -157.75666667) 
    v15 = (17.805,-156.06805556)
    v16 = (18.10888889, -155.71166667)
    v17 = (19.14222222, -154.48333333)
    v18 = (19.22293333, -151.87963333)
    v19 = (20.69694444, -151.01916667) 
    v20 = (21.54777778, -151.46638889)
    v21 = (22.34416667,-151.88527778)
    v22 = (23.02416667, -152.57777778)
    v23 = (23.78055556, -153.36611111)
    v24 = (24.29583333, -154.25)
    v25 = (24.72138889, -155.26305556)
    v26 = (25.19583333, -156.42111111)

    # Polygon
    coords = [v1, v2, v3, v4, v5, v6, v7, v8, v9, v10, v11, v12, v13, v14, v15, v16, v17, v18, v19, v20, v21, v22, v23, v24, v25, v26]
    poly = Polygon(coords)
    
    #Sort flights into what is in the airspace and what is not
    hawaiiAir = []

    for loc in range(0,len(airspaceData)):
      p1 = Point(airspaceData.iloc[loc][5], airspaceData.iloc[loc][6])
      hawaiiAir.append(p1.within(poly))

    airspaceData['nearHawaii'] = hawaiiAir
    
    #Filter out only the ones in the airspace
    airspaceData = airspaceData[(airspaceData['nearHawaii'] == False)]
    airspaceData = airspaceData.drop(columns=['nearHawaii'])
        

In [7]:
#Function to set the direction of aircraft

def aircraftDirection():
    #Set the dataframe that will be altered through this block of code
    global airspaceData
    
    # Replace missing value with -1
    airspaceData['SelectedHeading'] = airspaceData['SelectedHeading'].fillna(-1)
    
    # Assign Direction "E" for 0-180 degree, "W" for 180-360 degree, "NA" is record with null values 
    conditionlist = [
        (airspaceData['SelectedHeading'] < 0) ,
        (airspaceData['SelectedHeading'] >= 0) & (airspaceData['SelectedHeading'] <180),
        (airspaceData['SelectedHeading'] > 180)]
    choicelist = ['NA', 'E', 'W']
    airspaceData['Direction'] = np.select(conditionlist, choicelist)

In [8]:
#Function to control the execution order

def filterData():
    
    rawAircraftData = firstFiveSeconds()
    removeHISpace()
    aircraftDirection()
    
    return rawAircraftData

In [9]:
#Functions to call for the data cleanup

#Data set with filtered/clean data
airspaceData = filterAttributes()
#Raw data set with the time (H:M:S) broken out 
allAircraftData = filterData()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Filter out the minute intervals

In [10]:
def minuteFilter(HourCounter,MinuteCounter):

    global airspaceData

    #create SQL query for flights between the start and end time
    sql1 = "SELECT *, min(Second) FROM airspaceData WHERE Hour = '{0}' and Minute = '{1}' GROUP BY TargetID ORDER BY TargetID, Second".format(HourCounter, MinuteCounter)

    #Run query and store results
    recordsInMinute = pq.sqldf(sql1, globals())
    del recordsInMinute['min(Second)']

    return (recordsInMinute)

Proximity Calculation

In [11]:
def lonTableCreate(recordsByMinuteDF):
    #Create tables to check for proximity in latitude
    sql4 = "SELECT * FROM recordsByMinuteDF ORDER BY Longitude"

    #Run query and store results
    LongitudeOrderDF = pq.sqldf(sql4, locals())

    return (LongitudeOrderDF)

In [12]:
def latTableCreate(recordsByMinuteDF):
    #Create tables to check for proximity in latitude
    sql3 = "SELECT * FROM recordsByMinuteDF ORDER BY Latitude"

    #Run query and store results
    LatitudeOrderDF = pq.sqldf(sql3, locals())

    return (LatitudeOrderDF)

In [13]:
# Implement the formula below
def distance_d(point0,pointX):
    # The function "radians" is found in the math module
    LoA = radians(point0[1])  
    LoB = radians(pointX[1])
    LaA=  radians(point0[0])  
    LaB = radians(pointX[0]) 
    # The "Haversine formula" is used.
    D_Lo = LoB - LoA 
    D_La = LaB - LaA 
    P = sin(D_La / 2)**2 + cos(LaA) * cos(LaB) * sin(D_Lo / 2)**2  
   
    Q = 2 * asin(sqrt(P))   
    # The earth's radius in kilometers.
    R_km = 6371  
 
    # Change the kilometer to  nautical miles
    R_nm = R_km*0.539956803

    # Then we'll compute the outcome.
    return(Q * R_nm)

In [14]:
# Create function to set up boundary within 25 nm by latitude, longitude 
def limit_lon(point0):
    '''
    use with LongitudeOrderDF
    note: distance from point to longitude boundary of each row is around 24.9715
    '''
    LaA = radians(point0[0])
    onedeg_long = cos(LaA)*(69.172*0.868976242)
    add = 25/onedeg_long 
    pointlimit = (point0[0],point0[1]+add)
    return pointlimit[1]

def limit_lat(point0):
    '''
    use with LatitudeOrderDF
    note: distance from point to longitude boundary of each row is 25.016857125339488
    '''
    onedeg_lat = 60 
    add = 25/onedeg_lat
    pointlimit = (point0[0]+add,point0[1])
    return pointlimit[0]

In [15]:
# Create function to select, merge and add the values from analyzing Longitude and Latitude
def newDF(OrderDF,x,y,d):
    """DF is Long/LatitudeOrderDF
       x = long/latpoint_a
       y = long/latpoint_b
       d = long/latdistance_ab"""
    # select rows that index is in list 'point_a', 'point_b'
    A = OrderDF.loc[x,['DateTime','Hour','Minute','Second','microSecond','Latitude','Longitude','FlightLevel',
                             'TargetID', 'SelectedHeading', 'TargetAddress','Direction']]
    B = OrderDF.loc[y,['DateTime','Hour','Minute','Second','microSecond','Latitude','Longitude','FlightLevel',
                             'TargetID', 'SelectedHeading', 'TargetAddress','Direction']]
    # Join 2 tables by the "TargetID" of point a (for the uniquness)
    OrderResult = pd.merge(A.reset_index(drop=True),B.reset_index(drop=True),left_index=True, right_index=True)
    # add distance column
    OrderResult['Distance'] = d
    return OrderResult

In [16]:
#Calculate the distance of the points closest to each other by longitidue and latitude
def proximityCalc(LongitudeOrderDF, LatitudeOrderDF):
    longpoint_a = []
    longpoint_b = []
    longdistance_ab = []

    latpoint_a = []
    latpoint_b = []
    latdistance_ab = []

    for a in LongitudeOrderDF.index:
        for n in range(1,len(LongitudeOrderDF)):
            b = a+n
            if b < len(LongitudeOrderDF):
                point0 = LongitudeOrderDF.loc[a,'Latitude'], LongitudeOrderDF.loc[a,'Longitude']
                pointX = LongitudeOrderDF.loc[b,'Latitude'], LongitudeOrderDF.loc[b,'Longitude']
                if pointX[1] <= limit_lon(point0): # Check if longitude of pointX is within the boundary
                    distance = distance_d(point0,pointX)
                    if distance <= 25: # Check distance within 25 nm
                        longpoint_a.append(a)
                        longpoint_b.append(b)
                        longdistance_ab.append(distance)
                    else:
                        break

    for a in LatitudeOrderDF.index:   
        for n in range(1,len(LatitudeOrderDF)):
            b = a+n
            if b < len(LatitudeOrderDF):
                point0 = LatitudeOrderDF.loc[a,'Latitude'], LatitudeOrderDF.loc[a,'Longitude']
                pointX = LatitudeOrderDF.loc[b,'Latitude'], LatitudeOrderDF.loc[b,'Longitude']    
                if pointX[0] <= limit_lat(point0):# Check if latitude of pointX is within the boundary
                    distance = distance_d(point0,pointX)
                    if distance <= 25: # Check distance within 25 nm
                        latpoint_a.append(a)
                        latpoint_b.append(b)
                        latdistance_ab.append(distance)
                    else:
                        break
        
    # Apply function to select and merge data frame
    LongOrderResult = newDF(LongitudeOrderDF,longpoint_a, longpoint_b,longdistance_ab)
    LatOrderResult = newDF(LatitudeOrderDF,latpoint_a, latpoint_b,latdistance_ab)

    # Concatenate results from order by longitude and latitude
    Resultsdf = pd.concat([LongOrderResult,LatOrderResult]).reset_index(drop=True)

    # Delete duplicate pairs of TargetID x and y regardless of order
    Resultsdf['list_target'] = Resultsdf.apply(lambda row: tuple(sorted([row['TargetID_x']]+[row['TargetID_y']])), axis = 1)
    ResultsDF = Resultsdf.drop_duplicates(subset = 'list_target',keep = 'first').reset_index(drop = True)
    ResultsDF.drop('list_target', axis=1, inplace=True)

    return (ResultsDF)

Calculatin the height difference

In [17]:
def distanceCalc(resultsDF):
    heightDifference = []
    potentialLoss1000 = []
    potentialLoss400 = []

    for counter in range(0,len(resultsDF)):
        difference = abs((resultsDF['FlightLevel_x'][counter]) - (resultsDF['FlightLevel_y'][counter]))
        heightDifference.append(difference)

        if difference <= 1000:
            potentialLoss1000.append('True')
            if difference <= 400:
                potentialLoss400.append('True')
            else:
                potentialLoss400.append('False')
        else:
            potentialLoss1000.append('False')
            potentialLoss400.append('False')

    resultsDF['HeightDifference_ft'] = heightDifference
    resultsDF['potentialLoss400'] = potentialLoss400
    resultsDF['potentialLoss1000'] = potentialLoss1000

    return (resultsDF)

Compile the full list of results

In [18]:
finalResults = pd.DataFrame()

for HourCounter in range(0,1):
    #Create table for the minute
    for MinuteCounter in range(0,60):
        #Create table for the minute
        recordsByMinuteDF = minuteFilter(HourCounter,MinuteCounter)

        #Create longitude and latitude table for proximity analysis
        LongitudeOrderDF = lonTableCreate(recordsByMinuteDF)
        LatitudeOrderDF = latTableCreate(recordsByMinuteDF)

        #calculate proximity
        resultsDF = proximityCalc(LongitudeOrderDF, LatitudeOrderDF)
        if resultsDF.empty == True:
            # if the results dataframe is empty, then break out of for-loop
            break
        else:
            #Calculate distance
            resultsDF = distanceCalc(resultsDF)
            #Add the results for this minute to the overall results 
            finalResults = pd.concat([finalResults, resultsDF], ignore_index=True)
    HourCounter = HourCounter + 1

List of instances that had less than 400 height difference

In [19]:
#Get the first entry for this minute of time
sql6 = "SELECT * FROM finalResults WHERE potentialLoss400 = 'True' "

#Run query and store results
LossCandidates400 = pq.sqldf(sql6, locals())

print(LossCandidates400)

                   DateTime_x  Hour_x  Minute_x  Second_x  microSecond_x  \
0  2021-12-24 00:24:00.531000       0        24         0         531000   
1  2021-12-24 00:45:00.875000       0        45         0         875000   

   Latitude_x  Longitude_x  FlightLevel_x TargetID_x  SelectedHeading_x  ...  \
0   22.298813  -155.816895        30000.0     DAL495          45.703125  ...   
1   21.387222  -155.755821        30575.0    SWA1385         196.875000  ...   

  Longitude_y FlightLevel_y TargetID_y  SelectedHeading_y  TargetAddress_y  \
0 -155.472823       29800.0     AAL432          78.046875           AB271F   
1 -155.441362       30475.0    SWA1310          68.203125           ABFD8C   

   Direction_y   Distance  HeightDifference_ft  potentialLoss400  \
0            E  19.153655                200.0              True   
1            E  24.140767                100.0              True   

   potentialLoss1000  
0               True  
1               True  

[2 rows x 28 columns

New code to do analysis on the LossCandidates400 table
-----------------------------------------------------------

In [20]:
def recordsTable(x):
  flight_x = LossCandidates400['TargetID_x'][x]
  flight_y = LossCandidates400['TargetID_y'][x]
  hour = LossCandidates400['Hour_x'][x]
  minute = LossCandidates400['Minute_x'][x]

  flightInformation = allAircraftData.loc[((allAircraftData['TargetID'] == flight_x) | (allAircraftData['TargetID'] == flight_y)) & 
                                          ((allAircraftData['Minute'] >= (minute - 3)) & (allAircraftData['Minute'] <= (minute + 3))) & 
                                          ((allAircraftData['Hour'] == hour))]
  

  return flightInformation.sort_values(by=['TargetID','Minute', 'Second'])              


In [21]:
# function for fill missing second with linear interpolation
def fillSecond(data_x,data_y):
  '''This function transform data of target y to be 
      on the same minute, second as target x'''
  # filled with NA in data_y if second_x are not in second_y 
  Y = data_y.groupby('Minute')['Second'].apply(list).reset_index(name='list')
  for i in data_x.index:
    min_x = data_x.loc[i,'Minute']
    sec_x = data_x.loc[i,'Second']
    for n in range(0,len(Y)):
      min_y = Y.loc[n,'Minute']
      if min_x == min_y:
        listsec = Y.loc[n,'list']
        if (sec_x not in listsec):
          ydict = {'Minute': min_x, 'Second': sec_x, 
                   'TargetID': data_y.loc[i,'TargetID'],
                   'SelectedHeading': data_y.loc[i,'SelectedHeading']}
          data_y = data_y.append(ydict, ignore_index = True)

  # fill NA with linear interpolation method
  y_transformed = data_y.sort_values(by=['Minute','Second']).interpolate(method='linear')
  return y_transformed

In [22]:
def transformTable(flightData):
  for i, id in enumerate(flightData['TargetID'].unique()):
    if i == 0:
      data_x = flightData[(flightData['TargetID']== id)].reset_index(drop = True)
    else:
      data_y = flightData[(flightData['TargetID']== id)].reset_index(drop = True)

  data_x = data_x[['Minute','Second','Latitude','Longitude','FlightLevel','TargetID','SelectedHeading']]
  data_y = data_y[['Minute','Second','Latitude','Longitude','FlightLevel','TargetID','SelectedHeading']]

  y_transformed = fillSecond(data_x,data_y)

  analyzedTable = pd.merge(data_x,y_transformed,on=['Minute','Second'], how='left')

  return analyzedTable


In [23]:
analysisResults400 = pd.DataFrame()

x = 0 #Location -> this will eventually become a for loop to loop through all of the records, for now focusing on developing for one
#Get the data for the flight at +/- 3 minutes from when the loss of separation was flagged to be under 400 ft
flightData = recordsTable(x)
analyzedTable = transformTable(flightData)
analyzedTable

,Minute,Second,Latitude_x,Longitude_x,FlightLevel_x,TargetID_x,SelectedHeading_x,Latitude_y,Longitude_y,FlightLevel_y,TargetID_y,SelectedHeading_y
0,21,3,22.581848,-155.291449,32600.0,AAL432,78.046875,22.083497,-156.167196,27912.5,DAL495,45.703125
1,21,7,22.574111,-155.296122,32525.0,AAL432,78.046875,22.088548,-156.159007,28025.0,DAL495,45.703125
2,21,12,22.565127,-155.301514,32450.0,AAL432,78.046875,22.093692,-156.150685,28100.0,DAL495,45.703125
3,21,17,22.556468,-155.306702,32375.0,AAL432,78.046875,22.098976,-156.142089,28137.5,DAL495,45.703125
4,21,22,22.548706,-155.311374,32300.0,AAL432,78.046875,22.104120,-156.133728,28175.0,DAL495,45.703125
...,...,...,...,...,...,...,...,...,...,...,...,...
80,27,8,21.975952,-155.652449,24525.0,AAL432,78.046875,22.537614,-155.425949,31000.0,DAL495,45.703125
81,27,13,21.968327,-155.656942,24375.0,AAL432,78.046875,22.542828,-155.417379,31000.0,DAL495,45.703125
82,27,18,21.961158,-155.661163,24250.0,AAL432,78.046875,22.548368,-155.408249,31000.0,DAL495,45.703125
83,27,22,21.954803,-155.664934,24125.0,AAL432,78.046875,22.554192,-155.398631,31000.0,DAL495,45.703125


In [28]:
# Append the lateral Distance to table
def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):

    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))  * 0.539956803


analyzedTable['LateralDistance'] = \
    haversine(analyzedTable.Latitude_x, analyzedTable.Longitude_x,
                 analyzedTable.Latitude_y, analyzedTable.Longitude_y)

In [33]:
# Direction
conditionsX = [
    (analyzedTable.iloc[0]['Latitude_x'] - analyzedTable.iloc[-1]['Latitude_x'] < 0),
    (analyzedTable.iloc[0]['Latitude_x'] - analyzedTable.iloc[-1]['Latitude_x'] > 0)
    ]

# create a list of the values we want to assign for each condition
values = ['W', 'E']

# create a new column and use np.select to assign values to it using our lists as arguments
analyzedTable['X_direction'] = np.select(conditionsX, values)

conditionsY = [
    (analyzedTable.iloc[0]['Latitude_y'] - analyzedTable.iloc[-1]['Latitude_y'] < 0),
    (analyzedTable.iloc[0]['Latitude_y'] - analyzedTable.iloc[-1]['Latitude_y'] > 0)
    ]

# create a list of the values we want to assign for each condition
values = ['W', 'E']

# create a new column and use np.select to assign values to it using our lists as arguments
analyzedTable['Y_direction'] = np.select(conditionsY, values)

In [34]:
analyzedTable

,Minute,Second,Latitude_x,Longitude_x,FlightLevel_x,TargetID_x,SelectedHeading_x,Latitude_y,Longitude_y,FlightLevel_y,TargetID_y,SelectedHeading_y,LateralDistance,X_direction,Y_direction
0,21,3,22.581848,-155.291449,32600.0,AAL432,78.046875,22.083497,-156.167196,27912.5,DAL495,45.703125,57.102967,E,W
1,21,7,22.574111,-155.296122,32525.0,AAL432,78.046875,22.088548,-156.159007,28025.0,DAL495,45.703125,56.093319,E,W
2,21,12,22.565127,-155.301514,32450.0,AAL432,78.046875,22.093692,-156.150685,28100.0,DAL495,45.703125,55.003347,E,W
3,21,17,22.556468,-155.306702,32375.0,AAL432,78.046875,22.098976,-156.142089,28137.5,DAL495,45.703125,53.917652,E,W
4,21,22,22.548706,-155.311374,32300.0,AAL432,78.046875,22.104120,-156.133728,28175.0,DAL495,45.703125,52.901232,E,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,27,8,21.975952,-155.652449,24525.0,AAL432,78.046875,22.537614,-155.425949,31000.0,DAL495,45.703125,35.994561,E,W
81,27,13,21.968327,-155.656942,24375.0,AAL432,78.046875,22.542828,-155.417379,31000.0,DAL495,45.703125,36.972890,E,W
82,27,18,21.961158,-155.661163,24250.0,AAL432,78.046875,22.548368,-155.408249,31000.0,DAL495,45.703125,37.954193,E,W
83,27,22,21.954803,-155.664934,24125.0,AAL432,78.046875,22.554192,-155.398631,31000.0,DAL495,45.703125,38.911166,E,W
